In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


from astropy.io import fits



In [2]:
# Assembly task
# https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/isrTask.py

from lsst.ip.isr.assembleCcdTask import (AssembleCcdConfig, AssembleCcdTask)
from lsst.ip.isr.isrTask import (IsrTask, IsrTaskConfig)

#https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/overscan.py
from lsst.ip.isr import  OverscanCorrectionTaskConfig, OverscanCorrectionTask

In [3]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [4]:
import warnings
warnings.filterwarnings("ignore")

## Butler 

In [5]:
import lsst.daf.butler as dafButler

In [6]:
embargo = True

In [7]:
if embargo:
    repo="/sdf/group/rubin/repo/oga/"
else:
    repo = "/sdf/group/rubin/repo/main"
butler = dafButler.Butler(repo)
registry = butler.registry

In [8]:
calibCollections = ['LATISS/defaults','LATISS/raw/all']
for col in registry.queryCollections("*LATISS/calib*"):
    calibCollections.append(col)

In [9]:
where_flats = "instrument='LATISS' AND physical_filter='empty~empty' AND exposure.observation_type='flat'"

In [10]:
flats_records = list(registry.queryDimensionRecords('exposure',where=where_flats))

In [11]:
print(len(flats_records))
assert len(flats_records)!=0

666


In [13]:
flat_dates = []
for flat_ in flats_records:
    if flat_year in str(flat_.day_obs):
        flat_dates.append(flat_.day_obs)


Available dates for flats of a given year 

In [12]:
flat_year = '2023'

In [14]:
flat_dates = np.unique(flat_dates)
print(flat_dates)

[20230112 20230116 20230118 20230214 20230321 20230327 20230404 20230420
 20230425 20230509 20230510 20230523 20230524 20230525 20230705 20230706
 20230724 20230725 20230726 20230727 20230801 20230802 20230803 20230811
 20230816 20230817 20230823 20230829 20230830]


We select a date close to the that of the spectra that we want to deflat: 

In [15]:
flat_date = 20230116

Available flat ids 

In [20]:
flats_selected = []
for flat_ in flats_records:
    if flat_.day_obs==flat_date:
        flats_selected.append(flat_.id)
print('Flat IDs: ', flats_selected)

Flat IDs:  [2023011600015, 2023011600014, 2023011600016]


Check that we can load the flats 

In [19]:
flat_arrays = []
for fid_ in flats_selected:
    try:
        flat_img = butler.get(
            "flat",
            instrument="LATISS", exposure=fid_, detector=0,
            collections=calibCollections)
        flat_arrays.append(flat_img.getImage().array)
        print('Flat {0} loaded'.format(fid_))
    except:
        print('Could not load flat {0}'.format(fid_))

Flat 2023011600015 loaded
Flat 2023011600014 loaded
Flat 2023011600016 loaded
